# Información de Campos del dataframe

| Field   | Comment                                                                 |
|---------|-------------------------------------------------------------------------|
| START   | The start timestamp of the template execution in milliseconds           |
| END     | The end timestamp of the template execution in milliseconds             |
| TIMEOUT | Indicates if the execution exceeded a predefined time limit             |
| system  | The name of the system used (e.g., PIONIER)                             |
| procname| The process name associated with the template execution                 |
| TPL_ID  | The filename of the corresponding template file                         |
| ERROR   | Indicates if there was an error during execution                        |
| Aborted | Indicates if the template execution was aborted (manually or because an error) |
| SECONDS | The duration of the template execution in seconds                       |
| TEL     | The class of telescope used in the observation, in this dataset it is only AT |

# Información de Campos de Log (metadata de los Log)

| Field       | Description                                                                                           |
|-------------|-------------------------------------------------------------------------------------------------------|
| @timestamp  | The timestamp of the log entry in milliseconds.                                                       |
| system      | The name of the system (e.g., PIONIER) from which the log entry originates.                           |
| hostname    | The hostname of the machine where the log entry was generated.                                        |
| loghost     | The host of the logging system that generated the entry.                                              |
| logtype     | Type of the log entry (e.g., LOG, FEVT, ERR), indicating its nature such as general log, event, or error. |
| envname     | The environment name where the log was generated, providing context for the log entry.                |
| procname    | The name of the process that generated the log entry.                                                 |
| procid      | The process ID associated with the log entry.                                                         |
| module      | The module from which the log entry originated, indicating the specific part of the system.           |
| keywname    | Name of any keyword associated with the log entry, if applicable. It is always paired with keywvalue. |
| keywvalue   | Value of the keyword mentioned in keywname, if applicable.                                            |
| keywmask    | Mask or additional context for the keyword, if applicable.                                            |
| logtext     | The actual text of the log entry, providing detailed information about the event or action.           |
| trace_id    | A unique identifier associated with each log entry, corresponds to id in metadata table.              |

In [1]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Función para generar el código PlantUML para un solo usuario
def generate_plantuml_code_single_user(meta_row, trace_row):
    uml_code = "@startuml\n"
    uml_code += "actor User\n"
    uml_code += "participant System\n"
    uml_code += "participant Instrument\n"
    uml_code += "participant Subsystem\n\n"

    start_time = meta_row['START']
    end_time = meta_row['END']
    procname = meta_row['procname']
    status = 'success' if not meta_row['ERROR'] else 'failure'
    logtext = trace_row['logtext']

    uml_code += f"User -> System: Send Command ({procname})\n"
    uml_code += "activate System\n"
    uml_code += f"System -> Instrument: Forward Command ({procname})\n"
    uml_code += "activate Instrument\n"
    uml_code += f"Instrument -> Subsystem: Execute Command ({procname})\n"
    uml_code += "activate Subsystem\n\n"

    if status == 'success':
        uml_code += f"Subsystem -> Instrument: Return Success ({start_time}, {end_time})\n"
    else:
        uml_code += f"Subsystem -> Instrument: Return Failure ({start_time}, {end_time})\n"
    
    uml_code += "deactivate Subsystem\n\n"
    uml_code += f"Instrument -> System: Return Status ({status})\n"
    uml_code += "deactivate Instrument\n\n"
    uml_code += f"System -> User: Return Status ({status})\n"
    uml_code += "deactivate System\n\n"
    uml_code += f"note right of System: Log Entry - {logtext}\n"

    uml_code += "@enduml"
    return uml_code

# Cargar los datos de observación y metadatos
INSTRUMENT = "PIONIER"
RANGE = "1w"
TRACE_ID = 100  # Ejemplo de trace_id, ajusta según tus datos

df_meta = load_meta(INSTRUMENT, RANGE)
df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)

# Verificar las columnas de los DataFrames
print("Columnas en df_meta:", df_meta.columns)
print("Columnas en df_trace:", df_trace.columns)

# Seleccionar una fila del DataFrame para el usuario

meta_row=df_meta.iloc[TRACE_ID]


if 'trace_id' in df_trace.columns:
    trace_row = df_trace.iloc[0]
else:
    print("La columna 'trace_id' no está presente en df_trace. Verifica el nombre de la columna.")

# Generar el código PlantUML para un solo usuario
uml_code = generate_plantuml_code_single_user(meta_row, trace_row)

# Guardar el código PlantUML en un archivo
with open('dynamic_diagram.puml', 'w') as file:
    file.write(uml_code)

# Renderizar el diagrama usando PlantUML a través de subprocess
subprocess.run(['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'])

Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')


CompletedProcess(args=['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'], returncode=1)

In [2]:
from PIL import Image

In [3]:
df_meta.iloc[TRACE_ID]

START       2019-04-03 17:08:08.894000
END         2019-04-03 17:08:42.030000
TIMEOUT                          False
system                         PIONIER
procname                       bob_ins
TPL_ID         PIONIER_gen_tec_standby
ERROR                            False
Aborted                          False
SECONDS                           33.0
TEL                                 AT
Name: 100, dtype: object

In [4]:
trace_id=10

In [6]:
df_trace = load_trace("MATISSE", "1w", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]
df_tratamiento=df_trace[ df_trace['logtext'].str.contains("CHKMOON") ][:]
lista = ['Send','Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_system = []

for x in lista:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['system']
    Lista_system.extend(df_cc.tolist())

print(Lista_system)

lista1 = ['Send','Received command', 'Waiting', 'Succesfully completed', 'Last reply']

Lista_system = []

for x in lista1:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['procname']
    Lista_system.extend(df_cc.tolist())

print(Lista_system)


['MATISSE', 'ISS', 'MATISSE', 'ISS', 'MATISSE']
['bob_ins', 'issifControl', 'bob_ins', 'issifControl', 'bob_ins']


In [11]:
lista2 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_time = []
for x in lista2:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['@timestamp']
    Lista_procname.extend(df_cc.tolist())
Lista_procname

['bob_ins',
 'issifControl',
 'bob_ins',
 'issifControl',
 'bob_ins',
 Timestamp('2019-04-02 01:30:13.397000'),
 Timestamp('2019-04-02 01:30:13.418000'),
 Timestamp('2019-04-02 01:30:13.419000'),
 Timestamp('2019-04-02 01:30:13.482000'),
 Timestamp('2019-04-02 01:30:14.440000')]

# Secuencia de codigo utilizado para el codigo UML

In [15]:
df_meta = load_meta("MATISSE", "1w")
df_meta

,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,SECONDS,TEL
0,2019-04-02 01:29:45.681,2019-04-02 01:38:08.541,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,502.0,AT
1,2019-04-02 01:38:08.559,2019-04-02 01:44:02.506,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,353.0,AT
2,2019-04-02 01:45:44.711,2019-04-02 01:58:00.921,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,736.0,AT
3,2019-04-02 01:58:19.863,2019-04-02 02:14:00.315,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,940.0,AT
4,2019-04-02 06:36:28.631,2019-04-02 07:00:16.566,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,1427.0,AT
...,...,...,...,...,...,...,...,...,...,...
215,2019-04-07 21:20:07.817,2019-04-07 21:22:12.808,False,MATISSE,bob_ins,MATISSE_gen_tec_opd,False,False,124.0,AT
216,2019-04-07 21:22:25.249,2019-04-07 21:34:42.495,False,MATISSE,bob_ins,MATISSE_gen_tec_check_fringes,False,False,737.0,AT
217,2019-04-07 21:34:42.495,2019-04-07 21:39:06.173,False,MATISSE,bob_ins,MATISSE_gen_cal_bias,False,False,263.0,AT
218,2019-04-07 21:40:15.747,2019-04-07 21:42:02.235,False,MATISSE,bob_ins,MATISSE_gen_cal_bias,False,False,106.0,AT


In [16]:
df_trace = load_trace("MATISSE", "1w", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]

In [17]:
with pd.option_context('display.max_colwidth', None):
    display(df_trace[ df_trace['logtext'].str.contains("CHKMOON") ][:])

,@timestamp,system,procname,logtext,trace_id
182,2019-04-02 01:30:13.397,MATISSE,bob_ins,"Send command CHKMOON 064508.910,-164258.000,0.0 to process issifControl env. wvgvlti",0
183,2019-04-02 01:30:13.418,ISS,issifControl,"CDP : Received command: CHKMOON, Buffer: 064508.910,-164258.000,0.0",0
184,2019-04-02 01:30:13.419,MATISSE,bob_ins,Waiting (timeout 40000) for pending replies to command CHKMOON id cmd0,0
185,2019-04-02 01:30:13.482,ISS,issifControl,"CDP : Succesfully completed command: CHKMOON, Buffer: OK",0
194,2019-04-02 01:30:14.440,MATISSE,bob_ins,Last reply to command CHKMOON id cmd0 received,0


In [5]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Función para generar el código PlantUML para un solo usuario
def generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname, Lista_time):
    uml_code = "@startuml\n"
    uml_code += 'actor User\n'
    uml_code += f'participant {Lista_system[0]}_{Lista_procname[0]} as {Lista_system[0]}_{Lista_procname[0]}\n'
    uml_code += f'participant {Lista_system[1]}_{Lista_procname[1]} as {Lista_system[1]}_{Lista_procname[1]}\n\n'

    uml_code += f'User -> {Lista_system[0]}_{Lista_procname[0]}: Send Command CHKMOON\n'
    uml_code += f'note right of User: {str(Lista_time[0])}\n'
    uml_code += f'{Lista_system[0]}_{Lista_procname[0]} -> {Lista_system[1]}_{Lista_procname[1]}: CDP: Received command CHKMOON\n'
    uml_code += f'note right of {Lista_system[0]}_{Lista_procname[0]}: {str(Lista_time[1])}\n'
    uml_code += f'{Lista_system[1]}_{Lista_procname[1]} -> {Lista_system[1]}_{Lista_procname[1]}: Waiting (timeout 40000) for ...\n'
    uml_code += f'note right of {Lista_system[1]}_{Lista_procname[1]}: {str(Lista_time[2])}\n'
    uml_code += f'{Lista_system[1]}_{Lista_procname[1]} -> {Lista_system[0]}_{Lista_procname[0]}: CDP: Successfully completed command CHKMOON\n'
    uml_code += f'note right of {Lista_system[0]}_{Lista_procname[0]}: {str(Lista_time[3])}\n'
    uml_code += f'{Lista_system[0]}_{Lista_procname[0]} -> User: Last reply to command CHKMOON\n\n'
    uml_code += f'note right of User: {str(Lista_time[4])}\n'
    uml_code += "@enduml"
    return uml_code

# Cargar los datos de observación y metadatos
INSTRUMENT = "MATISSE"
RANGE = "1w"
TRACE_ID = 0  # Ejemplo de trace_id, ajusta según tus datos

df_meta = load_meta(INSTRUMENT, RANGE)
df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)

# Verificar las columnas de los DataFrames
print("Columnas en df_meta:", df_meta.columns)
print("Columnas en df_trace:", df_trace.columns)

# Seleccionar una fila del DataFrame para el usuario
meta_row = df_meta.iloc[TRACE_ID]

if 'trace_id' in df_trace.columns:
    trace_row = df_trace.iloc[0]
else:
    print("La columna 'trace_id' no está presente en df_trace. Verifica el nombre de la columna.")

# Filtrar y procesar los datos de df_trace
df_trace = load_trace("MATISSE", "1w", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]
df_tratamiento = df_trace[df_trace['logtext'].str.contains("CHKMOON")][:]

lista = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_system = []

for x in lista:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['system']
    Lista_system.extend(df_cc.tolist())

print(Lista_system)

lista1 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_procname = []

for x in lista1:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['procname']
    Lista_procname.extend(df_cc.tolist())

print(Lista_procname)

lista2 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_time = []
for x in lista2:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['@timestamp']
    Lista_time.extend(df_cc.tolist())

print(Lista_time)

# Lista que contiene el texto del log_text
#lista_log_text=df_trace[df_trace['logtext'].str.contains("CHKMOON")][:]['logtext'].tolist()


# Generar el código PlantUML para un solo usuario
uml_code = generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname, Lista_time)

# Guardar el código PlantUML en un archivo
with open('dynamic_diagram.puml', 'w') as file:
    file.write(uml_code)

# Renderizar el diagrama usando PlantUML a través de subprocess
subprocess.run(['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'])

print("Diagrama UML generado y guardado como 'dynamic_diagram.png'")


Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')
['MATISSE', 'ISS', 'MATISSE', 'ISS', 'MATISSE']
['bob_ins', 'issifControl', 'bob_ins', 'issifControl', 'bob_ins']
[Timestamp('2019-04-02 01:30:13.397000'), Timestamp('2019-04-02 01:30:13.418000'), Timestamp('2019-04-02 01:30:13.419000'), Timestamp('2019-04-02 01:30:13.482000'), Timestamp('2019-04-02 01:30:14.440000')]
Diagrama UML generado y guardado como 'dynamic_diagram.png'


In [54]:
# Filtrar el DataFrame y extraer el texto
filtered_text = df_trace[df_trace['logtext'].str.contains("CHKMOON")][:]['logtext'].tolist()

# Mostrar el texto extraído
for text in filtered_text:
    print(text)



Send command CHKMOON 064508.910,-164258.000,0.0 to process issifControl env. wvgvlti
CDP   : Received command: CHKMOON, Buffer: 064508.910,-164258.000,0.0
Waiting (timeout 40000) for pending replies to command CHKMOON id cmd0
CDP   : Succesfully completed command: CHKMOON, Buffer: OK
Last reply to command CHKMOON id cmd0 received


@startuml

actor User

participant MATISSE_bob_ins as MATISSE_bob_ins

participant ISS_issifControl as ISS_issifControl

User -> MATISSE_bob_ins: Send Command CHKMOON

note right of User: 2019-04-02 01:30:13.397000

MATISSE_bob_ins -> ISS_issifControl: CDP: Received command CHKMOON

note right of MATISSE_bob_ins: 2019-04-02 01:30:13.418000

ISS_issifControl -> ISS_issifControl: Waiting (timeout 40000) for ...

note right of ISS_issifControl: 2019-04-02 01:30:13.419000

ISS_issifControl -> MATISSE_bob_ins: CDP: Successfully completed command CHKMOON

note right of User: 2019-04-02 01:30:13.482000

MATISSE_bob_ins -> User: Last reply to command CHKMOON


@enduml

In [66]:
df_trace = load_trace("GRAVITY", "1w", 0)

In [67]:
cmd = df_trace[ df_trace['logtext'].str.contains("command")]['logtext']

# Please replace this with an elegant regexp!
cmd = cmd.apply(lambda x: x.replace(":", " "))
cmd = cmd.apply(lambda x: x.replace(",", " "))
cmd = cmd.apply(lambda x: x.replace("'", " "))
cmd = cmd.apply(lambda x: x.replace("  ", " "))

cmdlist = []
for x in cmd:
    splitted = x.split()
    # print(splitted)
    if "command" in splitted:
        idx = splitted.index("command")
        command = splitted[idx+1]
        if command not in ["...", "done.", "to"]:
            cmdlist.append(command)
set(cmdlist)

{'(old',
 '0',
 '6',
 'ADRCORR',
 'ATTACH',
 'BLINK',
 'CALIBSN',
 'CLOSELP',
 'CLRMODE',
 'CLRSTP',
 'CONFMOD',
 'CONTROL',
 'CONVABS',
 'CONVREL',
 'ENABLE',
 'GETDARK',
 'GETDELD',
 'GETIAS',
 'MODBLCK',
 'OFFGFSM',
 'OFFSACQ',
 'OPENLP',
 'PRABS',
 'PRCHK',
 'PRESET',
 'PRSNAME',
 'PRTOGS',
 'READOFF',
 'RESAMP',
 'RESFREQ',
 'RESREF',
 'ROOFPOS',
 'SETAMP',
 'SETBEAM',
 'SETCONS',
 'SETFIW',
 'SETGAIN',
 'SETGFSM',
 'SETGVCM',
 'SETHK',
 'SETLPUP',
 'SETMODE',
 'SETOFF',
 'SETPNT',
 'SETPOS',
 'SETPTFI',
 'SETREF',
 'SETSTAT',
 'SETSTEP',
 'SETTRAJ',
 'SETUP',
 'STOPAG',
 'STOPCAG',
 'STOPLP',
 'STORSKY',
 'STRTLP',
 'STRTSAF',
 'TRACKOF',
 'TRACKON',
 'is',
 'lgvics2/ic0lcuServer'}

In [47]:
unique_cmds = set(cmdlist)
list(unique_cmds)



['STOPLAG',
 'SETREF',
 'GETDEL',
 'SETPOF',
 'GETDELD',
 'STOPAG',
 'STOPCAG',
 'SETBEAM',
 'PRABS',
 'SETTADC',
 'SETUP',
 'PRSNAME',
 'PRTOGS',
 'GETCONF',
 'PRCHK',
 'STRTLAG',
 'STOPCHP',
 'PRESET',
 'PRSCOOR',
 'CLRSTP',
 'CHKMOON',
 'GETIAS']

In [5]:
with pd.option_context('display.max_colwidth', None):
    display(df_trace[ df_trace['logtext'].str.contains("CLRSTP") ][:])

,@timestamp,system,procname,logtext,trace_id
241,2019-04-02 01:30:17.668,MATISSE,bob_ins,Send command CLRSTP to process issifControl env. wvgvlti,0
242,2019-04-02 01:30:17.670,MATISSE,bob_ins,Waiting (timeout 15000) for pending replies to command CLRSTP id cmd0,0
243,2019-04-02 01:30:17.670,ISS,issifControl,"CDP : Received command: CLRSTP, Buffer:",0
244,2019-04-02 01:30:17.672,AT1,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
245,2019-04-02 01:30:17.672,AT1,atifControl,"TIF : Received command: CLRSTP, Buffer:",0
246,2019-04-02 01:30:17.673,AT2,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
247,2019-04-02 01:30:17.673,AT2,atifControl,"TIF : Received command: CLRSTP, Buffer:",0
248,2019-04-02 01:30:17.675,AT3,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
249,2019-04-02 01:30:17.675,AT3,atifControl,"TIF : Received command: CLRSTP, Buffer:",0
250,2019-04-02 01:30:17.676,AT4,atifControl,"TIF : Received command: CLRSTP, Buffer:",0


In [49]:
# Filtrar el DataFrame y extraer el texto
filtered_text = df_trace[df_trace['logtext'].str.contains("CLRSTP")]['logtext'].tolist()

# Mostrar el texto extraído
for text in filtered_text:
    print(text)

#filtered_text

Send command CLRSTP  to process issifControl env. wvgvlti
Waiting (timeout 15000) for pending replies to command CLRSTP id cmd0
CDP   : Received command: CLRSTP, Buffer:
PRESET: Received command: CLRSTP, Buffer:
TIF   : Received command: CLRSTP, Buffer:
PRESET: Received command: CLRSTP, Buffer:
TIF   : Received command: CLRSTP, Buffer:
PRESET: Received command: CLRSTP, Buffer:
TIF   : Received command: CLRSTP, Buffer:
TIF   : Received command: CLRSTP, Buffer:
PRESET: Received command: CLRSTP, Buffer:
PRESET: Received command: CLRSTP, Buffer:
CONTROL: Received command: CLRSTP, Buffer:
TIF   : Received command: CLRSTP, Buffer:
PRESET: Command CLRSTP (NoMove) succesfully completed
TIF   : Succesfully completed command: CLRSTP, Buffer: OK
CONTROL: Command CLRSTP (NoMove) succesfully completed
TIF   : Succesfully completed command: CLRSTP, Buffer: OK
TIF   : Succesfully completed command: CLRSTP, Buffer: OK
TIF   : Succesfully completed command: CLRSTP, Buffer: OK
TIF   : Succesfully comple

In [7]:
# Filtrar el DataFrame
filtered_df = df_trace[df_trace['logtext'].str.contains("CLRSTP")][:]
# Guardar el DataFrame filtrado en un archivo CSV
filtered_df.to_csv('filtered_trace_CLRSTP.csv', index=False)

In [29]:
with pd.option_context('display.max_colwidth', None):
    display(df_trace[ df_trace['logtext'].str.contains("SETUP") ][:])

,@timestamp,system,procname,logtext,trace_id
130,2019-04-02 01:29:47.434,MATISSE,bob_ins,SETUP -noExposure -function DET1.START.VAL 1 DET2.START.VAL 1 (blue),0
131,2019-04-02 01:29:47.435,MATISSE,mtoControl,Executing SETUP command ...,0
132,2019-04-02 01:29:47.436,MATISSE,mtoControl,Forward(b) SETUP to NGCIR1,0
133,2019-04-02 01:29:47.436,MATISSE,mtoControl,Forward(b) SETUP to NGCIR2,0
135,2019-04-02 01:29:47.440,MATISSE,mtoControl,SETUP command done.,0
...,...,...,...,...,...
2359,2019-04-02 01:38:00.099,MATISSE,mtoControl,Executing SETUP command ...,0
2373,2019-04-02 01:38:00.488,MATISSE,mtoControl,SETUP command done.,0
2374,2019-04-02 01:38:00.489,MATISSE,bob_ins,SETUP -noExposure -function INS.SOS1.NAME OUT INS.SOS2.NAME OUT INS.BCD1.NAME OUT INS.BCD2.NAME OUT INS.OML1.OPD 0 INS.OML2.OPD 0 INS.OML3.OPD 0 INS.OML4.OPD 0,0
2377,2019-04-02 01:38:00.489,MATISSE,mtoControl,Executing SETUP command ...,0


In [37]:
# Filtrar el DataFrame
filtered_df = df_trace[df_trace['logtext'].str.contains("SETUP")][:]

# Guardar el DataFrame filtrado en un archivo CSV
filtered_df.to_csv('filtered_trace_setup.csv', index=False)

In [48]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Función para generar el código PlantUML para un solo usuario
def generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname, Lista_time, lista_log_text):
    uml_code = "@startuml\n"
    uml_code += 'actor User\n'
    
    # Crear un diccionario para rastrear los participantes únicos y sus procname
    participants = {}
    
    # Agregar la primera transición del User al primer sistema
    if Lista_system:
        first_system = Lista_system[0]
        first_procname = Lista_procname[0]
        uml_code += f'participant {first_system}\n'
        uml_code += f'User -> {first_system}: {lista_log_text[0]}\n'
        uml_code += f'note right of User: {str(Lista_time[0])}\n'
        uml_code += f'note right of User: {lista_log_text[0]}\n'
        participants[first_system] = [first_procname]
    
    # Generar las interacciones entre los sistemas y agregar las notas de log_text
    for i in range(len(Lista_system)):
        system = Lista_system[i]
        procname = Lista_procname[i]
        
        # Agregar el participante si no ha sido agregado antes
        if system not in participants:
            participants[system] = [procname]
            uml_code += f'participant {system}\n'
        else:
            # Si el procname no está en la lista de procname del sistema, agregarlo
            if procname not in participants[system]:
                participants[system].append(procname)
                uml_code += f'note right of {system}: {procname}\n'
        
        from_participant = system
        if i < len(Lista_system) - 1:
            next_system = Lista_system[i+1]
            next_procname = Lista_procname[i+1]
            to_participant = next_system
            if system != next_system:
                uml_code += f'{from_participant} -> {to_participant}: {lista_log_text[i]}\n'
                uml_code += f'note right of {from_participant}: {str(Lista_time[i])}\n'
                uml_code += f'note right of {from_participant}: {lista_log_text[i]}\n'
            else:
                uml_code += f'note right of {from_participant}: {str(Lista_time[i])}\n'
                uml_code += f'note right of {from_participant}: {lista_log_text[i]}\n'
        else:
            uml_code += f'{from_participant} -> User: {lista_log_text[i]}\n'
            uml_code += f'note right of {from_participant}: {str(Lista_time[i])}\n'
            uml_code += f'note right of {from_participant}: {lista_log_text[i]}\n'

    uml_code += "@enduml"
    return uml_code

# Cargar los datos de observación y metadatos
INSTRUMENT = "MATISSE"
RANGE = "6m"
TRACE_ID = 4  # Ejemplo de trace_id, ajusta según tus datos

df_meta = load_meta(INSTRUMENT, RANGE)
df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)

# Verificar las columnas de los DataFrames
print("Columnas en df_meta:", df_meta.columns)
print("Columnas en df_trace:", df_trace.columns)

# Seleccionar una fila del DataFrame para el usuario
meta_row = df_meta.iloc[TRACE_ID]

if 'trace_id' in df_trace.columns:
    trace_row = df_trace.iloc[0]
else:
    print("La columna 'trace_id' no está presente en df_trace. Verifica el nombre de la columna.")

# Filtrar y procesar los datos de df_trace

df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]
df_tratamiento = df_trace[df_trace['logtext'].str.contains("CLRSTP")][:]

lista = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_system = []

for x in lista:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['system']
    Lista_system.extend(df_cc.tolist())

print(Lista_system)

lista1 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_procname = []

for x in lista1:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['procname']
    Lista_procname.extend(df_cc.tolist())

print(Lista_procname)

lista2 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_time = []
for x in lista2:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['@timestamp']
    Lista_time.extend(df_cc.tolist())

print(Lista_time)

lista_log_text = df_trace[df_trace['logtext'].str.contains("CLRSTP")][:]['logtext'].tolist()

# Generar el código PlantUML para un solo usuario
uml_code = generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname, Lista_time, lista_log_text)

# Guardar el código PlantUML en un archivo
with open('dynamic_diagram.puml2', 'w') as file:
    file.write(uml_code)

# Renderizar el diagrama usando PlantUML a través de subprocess
subprocess.run(['java', '-jar', 'plantuml.jar', 'dynamic_diagram2.puml'])

print("Diagrama UML generado y guardado como 'dynamic_diagram2.png'")

Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')
['GRAVITY', 'ISS', 'AT1', 'AT1', 'AT2', 'AT2', 'AT3', 'AT4', 'AT4', 'AT3', 'DL', 'ISS', 'DL', 'GRAVITY', 'AT1', 'AT3', 'AT4', 'DL', 'AT2', 'ISS', 'GRAVITY']
['bob_26244', 'issifControl', 'prsControl', 'atifControl', 'atifControl', 'prsControl', 'atifControl', 'atifControl', 'prsControl', 'prsControl', 'dlifControl', 'issprsControl', 'dlprsControl', 'bob_26244', 'atifControl', 'atifControl', 'atifControl', 'dlifControl', 'atifControl', 'issifControl', 'bob_26244']
[Timestamp('2019-04-02 02:26:47.918000'), Timestamp('2019-04-02 02:26:47.921000'), Timestamp('2019-04-02 02:26:47.922000'), Timestamp('2019-04-02 02:2

In [58]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

# URL of the repository containing the datasets
REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

# Determine the path based on the environment (Colab or local)
if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

# Function to load the dataset for a given instrument and range
def load_dataset(INSTRUMENT, RANGE):
    # Load instrument traces
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    # Load subsystem traces
    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    # Load telescope traces
    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    # Combine all traces into a single DataFrame
    all_traces = [df_inst, df_subs, df_tele]
    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

# Function to load a specific trace by trace_id
def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

# Function to load metadata for a given instrument and range
def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Function to generate PlantUML code for a single user
def generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname, Lista_time, lista_log_text):
    uml_code = "@startuml\n"
    uml_code += 'actor User\n'
    
    # Create a dictionary to track unique participants and their procname
    participants = {}
    
    # Add the first transition from User to the first system
    if Lista_system:
        first_system = Lista_system[0]
        first_procname = Lista_procname[0]
        uml_code += f'participant {first_system}\n'
        uml_code += f'User -> {first_system}: {lista_log_text[0]}\n'
        uml_code += f'note right of User: {str(Lista_time[0])}\n'
        uml_code += f'note right of User: {lista_log_text[0]}\n'
        participants[first_system] = [first_procname]
    
    # Generate interactions between systems and add log_text notes
    for i in range(len(Lista_system)):
        system = Lista_system[i]
        procname = Lista_procname[i]
        
        # Add the participant if not already added
        if system not in participants:
            participants[system] = [procname]
            uml_code += f'participant {system}\n'
        else:
            # If procname is not in the system's procname list, add it
            if procname not in participants[system]:
                participants[system].append(procname)
                uml_code += f'note right of {system}: {procname}\n'
        
        from_participant = system
        if i < len(Lista_system) - 1:
            next_system = Lista_system[i+1]
            next_procname = Lista_procname[i+1]
            to_participant = next_system
            if system != next_system:
                uml_code += f'{from_participant} -> {to_participant}: {lista_log_text[i]}\n'
                uml_code += f'note right of {from_participant}: {str(Lista_time[i])}\n'
                uml_code += f'note right of {from_participant}: {lista_log_text[i]}\n'
            else:
                uml_code += f'note right of {from_participant}: {str(Lista_time[i])}\n'
                uml_code += f'note right of {from_participant}: {lista_log_text[i]}\n'
        else:
            uml_code += f'{from_participant} -> User: {lista_log_text[i]}\n'
            uml_code += f'note right of {from_participant}: {str(Lista_time[i])}\n'
            uml_code += f'note right of {from_participant}: {lista_log_text[i]}\n'

    uml_code += "@enduml"
    return uml_code

# Function to visualize the command sequence
def visualization(INSTRUMENT, RANGE, TRACE_ID, COMMANDLIST):
    df_meta = load_meta(INSTRUMENT, RANGE)
    df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)
    
    # Verify the columns of the DataFrames
    print("Columns in df_meta:", df_meta.columns)
    print("Columns in df_trace:", df_trace.columns)
    
    # Select a row from the DataFrame for the user
    meta_row = df_meta.iloc[TRACE_ID]

    if 'trace_id' in df_trace.columns:
        trace_row = df_trace.iloc[0]
    else:
        print("The column 'trace_id' is not present in df_trace. Verify the column name.")

    # Filter and process the data from df_trace
    df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]
    df_tratamiento = df_trace[df_trace['logtext'].str.contains(COMMANDLIST)][:]

    lista = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
    Lista_system = []

    for x in lista:
        df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['system']
        Lista_system.extend(df_cc.tolist())

    print(Lista_system)

    lista1 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
    Lista_procname = []

    for x in lista1:
        df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['procname']
        Lista_procname.extend(df_cc.tolist())

    print(Lista_procname)

    lista2 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
    Lista_time = []
    for x in lista2:
        df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['@timestamp']
        Lista_time.extend(df_cc.tolist())

    print(Lista_time)

    lista_log_text = df_trace[df_trace['logtext'].str.contains(COMMANDLIST)][:]['logtext'].tolist()
    
    # Generate PlantUML code for a single user
    uml_code = generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname, Lista_time, lista_log_text)

    # Save the PlantUML code to a file
    with open('dynamic_diagram.puml3', 'w') as file:
        file.write(uml_code)

    print("UML diagram generated and saved as 'dynamic_diagram3.png'")

    return subprocess.run(['java', '-jar', 'plantuml_jar', uml_code], capture_output=True, text=True)  # Render the diagram using PlantUML through subprocess

In [68]:
visualization("PIONIER", "1M", 0, "STATUS")

Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')
['PIONIER']
['pnoControl']
[Timestamp('2019-04-01 22:29:07.859000')]
Diagrama UML generado y guardado como 'dynamic_diagram3.png'


CompletedProcess(args=['java', '-jar', ' plantuml_jar', '@startuml\nactor User\nparticipant PIONIER\nUser -> PIONIER: STATUS -function INS.MODE (blue)\nnote right of User: 2019-04-01 22:29:07.859000\nnote right of User: STATUS -function INS.MODE (blue)\nPIONIER -> User: STATUS -function INS.MODE (blue)\nnote right of PIONIER: 2019-04-01 22:29:07.859000\nnote right of PIONIER: STATUS -function INS.MODE (blue)\n@enduml'], returncode=1, stdout='', stderr='Error: Unable to access jarfile  plantuml_jar\n')

In [2]:
df_tratamiento

,@timestamp,system,procname,logtext,trace_id
241,2019-04-02 01:30:17.668,MATISSE,bob_ins,Send command CLRSTP to process issifControl e...,0
242,2019-04-02 01:30:17.670,MATISSE,bob_ins,Waiting (timeout 15000) for pending replies to...,0
243,2019-04-02 01:30:17.670,ISS,issifControl,"CDP : Received command: CLRSTP, Buffer:",0
244,2019-04-02 01:30:17.672,AT1,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
245,2019-04-02 01:30:17.672,AT1,atifControl,"TIF : Received command: CLRSTP, Buffer:",0
246,2019-04-02 01:30:17.673,AT2,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
247,2019-04-02 01:30:17.673,AT2,atifControl,"TIF : Received command: CLRSTP, Buffer:",0
248,2019-04-02 01:30:17.675,AT3,prsControl,"PRESET: Received command: CLRSTP, Buffer:",0
249,2019-04-02 01:30:17.675,AT3,atifControl,"TIF : Received command: CLRSTP, Buffer:",0
250,2019-04-02 01:30:17.676,AT4,atifControl,"TIF : Received command: CLRSTP, Buffer:",0
